In [ ]:
import cv2
import numpy as np
import math
import matplotlib.pyplot as plt
from gaze_tracking import GazeTracking


    
gaze = GazeTracking()
webcam = cv2.VideoCapture(0)

BODY_PARTS = { "Nose": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
               "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
               "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "REye": 14,
               "LEye": 15, "REar": 16, "LEar": 17, "Background": 18 }

POSE_PAIRS = [ ["Neck", "RShoulder"], ["Neck", "LShoulder"], ["RShoulder", "RElbow"],
               ["RElbow", "RWrist"], ["LShoulder", "LElbow"], ["LElbow", "LWrist"],
               ["Neck", "RHip"], ["RHip", "RKnee"], ["RKnee", "RAnkle"], ["Neck", "LHip"],
               ["LHip", "LKnee"], ["LKnee", "LAnkle"], ["Neck", "Nose"], ["Nose", "REye"],
               ["REye", "REar"], ["Nose", "LEye"], ["LEye", "LEar"] ]

inWidth = 368
inHeight = 368

net = cv2.dnn.readNetFromTensorflow("graph_opt.pb")

cap = cv2.VideoCapture(0)
time_series=[]
while True:
        # We get a new frame from the webcam
     _, frame = webcam.read()
    
        # We send this frame to GazeTracking to analyze it
     gaze.refresh(frame)
    
     frame = gaze.annotated_frame()
     text = ""
    

     if gaze.is_center():#and not gaze.is_bottom() and not gaze.is_top():
            if gaze.is_top():
                text = "Looking Top"
            elif gaze.is_bottom():
                text = "Looking Bottom"
            else:
                text = "Looking center"
     elif gaze.is_bottom() and gaze.is_right():
            text = "Looking Bottom Right"
     elif gaze.is_bottom() and gaze.is_left():
            text = "Looking Bottom Left"
     elif gaze.is_bottom():
            text = "Looking bottom"
     elif gaze.is_top():
            text = "Looking top"            
     elif gaze.is_top() and gaze.is_right():
            text = "Looking Top Right"
     elif gaze.is_bottom() and gaze.is_right():
            text = "Looking Bottom Right"
     elif gaze.is_top() and gaze.is_left():
            text = "Looking Top Left"
     elif gaze.is_right():
            text = "Looking right"
     elif gaze.is_left():
            text = "Looking left"
       
        
    
     cv2.putText(frame, text, (90,60), cv2.FONT_HERSHEY_DUPLEX, 1.6, (147,58,31), 2)
    
     left_pupil = gaze.pupil_left_coords()
     right_pupil = gaze.pupil_right_coords()
     vert_ratio = gaze.vertical_ratio()
     horz_ratio = gaze.horizontal_ratio()
     ##cv2.putText(frame, "Left pupil:  " + str(left_pupil), (10 ,130), cv2.FONT_HERSHEY_DUPLEX, 0.95, (255, 255, 255), 1)
     #cv2.putText(frame ,"Right pupil: " + str(right_pupil), (10 ,165), cv2.FONT_HERSHEY_DUPLEX, 0.95, (255, 255, 255), 1)
     cv2.putText(frame ,"Vertical Ratio " + str(vert_ratio), (10 ,200), cv2.FONT_HERSHEY_DUPLEX, 0.95, (147,58,31), 1)
     cv2.putText(frame ,"Horizontal Ratio " + str(horz_ratio), (10, 235), cv2.FONT_HERSHEY_DUPLEX, 0.95, (147,58,31), 1)
     #cv2.imshow("Tester 1 callibration", frame)
     #########################################################
     frameWidth = frame.shape[1]
     frameHeight = frame.shape[0]
    
     net.setInput(cv2.dnn.blobFromImage(frame, 1.0, (inWidth, inHeight), (127.5, 127.5, 127.5), swapRB=True, crop=False))
     out = net.forward()
     out = out[:, :19, :, :]

     assert(len(BODY_PARTS) == out.shape[1])
     
     points = []
     for i in range(len(BODY_PARTS)):
         # Slice heatmap of corresponging body's part.
         heatMap = out[0, i, :, :]
         _, conf, _, point = cv2.minMaxLoc(heatMap)
         x = (frameWidth * point[0]) / out.shape[3]
         y = (frameHeight * point[1]) / out.shape[2]
         # Add a point if it's confidence is higher than threshold.
         points.append((int(x), int(y)) if conf > 0.2 else None)

     for pair in POSE_PAIRS:
         partFrom = pair[0]
         partTo = pair[1]
         assert(partFrom in BODY_PARTS)
         assert(partTo in BODY_PARTS)

         idFrom = BODY_PARTS[partFrom]
         idTo = BODY_PARTS[partTo]

         if points[idFrom] and points[idTo]:
             cv2.line(frame, points[idFrom], points[idTo], (0, 255, 0), 3)
             cv2.ellipse(frame, points[idFrom], (3, 3), 0, 0, 360, (0, 0, 255), cv2.FILLED)
             cv2.ellipse(frame, points[idTo], (3, 3), 0, 0, 360, (0, 0, 255), cv2.FILLED)
     def angle_between_points(point1, point2):
         if (point1 != None and point2 != None):
            x1, y1 = point1
            x2, y2 = point2
            return math.degrees(math.atan2(y2 - y1, x2 - x1))
         else:
            return 0
     t, _ = net.getPerfProfile()
     freq = cv2.getTickFrequency() / 1000
     cv2.putText(frame, str(angle_between_points(points[5],points[6])), (90,120), cv2.FONT_HERSHEY_DUPLEX, 1.6, (147,58,31), 2)
     behave= ""
     if (gaze.is_left() and not gaze.is_top()) or (gaze.is_right() and not gaze.is_top()) :
        behave = "Abnormal eyes"
        time_series.append("0")
     elif angle_between_points(points[2],points[3]) > 160 or angle_between_points(points[2],points[3])<90 or angle_between_points(points[5],points[6]) < 30 or angle_between_points(points[5],points[6]) > 90:
        behave = "abnormal body"
        time_series.append("0")
     else:
        behave = "Normal"
        time_series.append("1")
     cv2.putText(frame, behave, (90,180), cv2.FONT_HERSHEY_DUPLEX, 1.6, (147,58,31), 2)
     cv2.imshow('OpenPose using OpenCV', frame)
     if cv2.waitKey(5) == 27:
        break

webcam.release()
cv2.destroyAllWindows()
print(time_series)
time_values = np.arange(len(time_series))
plt.plot(time_values, time_series)
plt.title("Time Series Graph")
plt.xlabel("Time")
plt.ylabel("Value")
plt.show()